# RAG Apps

## ¿Que son las RAG Apps?

Las RAG (Retrieval Augmented Generation) Apps son aplicaciones que nos permiten ajustar modelos de lenguajes a un contexto en especifico, en este caso lo que se busca es tener un documento que le agregara contexto a nuestro modelo para que sea un poco mas conciente sobre de lo que se esta hablando. 
¿Por que se hace esto?, aunque los datos de entrenamiento de un modelo son grandisimos, puede que nosotros estemos interesados en algo muy especifico, como un local o algun tema sobre el que no ha sido entrenado, o tal vez lo que buscamos es que se comporte de tal manera, que pareciera que estamos charlando con el documento.

## Para refrescar la memoria

### Que son los embeddings

Son representaciones vectoriales densas de datos (estos normalmente son palabras, frases, parrafos e incluso en este caso, documentos) en un espacio de dimensión reducida que capturan las caracteristicas semanticas y sintacticas de las entidades que representan.

- Densidad
    El vector tiene valor en cada una de sus dimensiones.
- Dimensionalidad reducida
    Los vectores tienen una dimensionalidad fija, menor al vocabulario conocido.
- Distribucion semantica
    Se organizan de tal manera que aquellos que son similares, se encuentran mas cercanos entre si.

Podemos ver una representacion mas grafica a traves del siguiente link: [Embedding Projector](https://projector.tensorflow.org/)

## ¿Como es el proceso?

<center><img src="Imagenes/VectoreStore.png" width="750" height="375" ></center>

1. **Load :** Este proceso comienza desde que se cargan los documentos. Los documentos que son cargados seran utilizados como contexto para nuestra aplicacion.
2. **Chunking :** Una de las partes mas importantes del proceso es el troceo o chunking, durante este proceso lo que se busca es dividir el documento en pedazos mas pequeños para posteriarmente pasarlas por un modelo especializado en embeddings y que sean almacenados.
Existen diferentes metodos para realizar el chunking:
    - Fixed size (Tamaño fijo).
    - Recursive (Recursivo).
    - Document specific (Centrado en tipo de documento).
    - Semantic based (Basado en relevancia semantica).
        - Consultando el siguiente [link](https://huggingface.co/spaces/m-ric/chunk_visualizer) puedes visualizar de manera mas grafica algunos metodos de chunking.
3. **Embedding :** Los trozos generados se pasan por un modelo de embeddings, que como se menciono antes, son representaciones vectoriales que pueden colocarse de tal manera, que estos esten mas cerca entre si, segun su similitud.
4. **Store :** Los embeddings se almacenan en una base vectorial, estos permaneceran aqui hasta el momento en que sean llamados en una consulta para agregar valor contextual en la respuesta.


<center><img src="Imagenes/RAG App.png" width="750" height="375"></center>

5. **Question :** Una vez realizado todo el proceso de almacenado y procesado de la informacion contextual, pasamos al proceso de la generacion de una respuesta, iniciando por el procesamiento de la pregunta.

6. **Retrieve :** Despues de procesar la pregunta, el programa accede a la base de datos vectoriales y recupera los chunks con un valor contextual mas valiosos, para que estos en una etapa posterior sean utilizados.

7. **Prompt :** El flujo del programa se dirige al prompt, donde el modelo verifica el comportamiento que este debe tomar.

8. **LLM :** Tanto el prompt, como los chunks, son anexados para que el modelo comienza con la generacion de una respuesta.

9. **Answer :** Finalmente termianos en la obtencion de una respuesta contextualizada gracias al documento anexado al principio.

A continuacion se muestra un ejemplo sencillo de una RAG App:

In [2]:
!pip install asyncio -q
!pip install nest_ansyncio -q
!pip install langchain_community -q
!pip install langchain_text_splitters -q
!pip install langchain_ollama -q
!pip install langchain_chroma -q
!pip install langchain_core -q
!pip install IPython -q


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement nest_ansyncio (from versions: none)

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for nest_ansyncio

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe

In [3]:
import asyncio
import nest_asyncio
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from IPython.display import clear_output


USER_AGENT environment variable not set, consider setting it to identify your requests.


Durante esta fase, se establece la informacion que aportara valor contextual.

In [4]:
# Habilitar el soporte async en Jupyter
nest_asyncio.apply()

# Carga y divide los documentos
loader = WebBaseLoader("https://cgsait.udg.mx/es/cads#:~:text=LEO%20%C3%81TROX%2C%20superc%C3%B3mputo%20a%20tu,poder%20de%20procesamiento%20de%20datos.&text=150%20nodos%20c%C3%B3mputo%2C%20con%20un,c%C3%B3mputo%20con%20tecnolog%C3%ADa%20XEON%20PHI%20.")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Configuración de embeddings y vectorstore
local_embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

Establecemos el comportamiento del modelo (El modelo que se estara usando, definicion de un prompt template, anexamos la informacion contextual y creamos la cadena).

In [5]:
# Configuración del modelo de lenguaje y plantilla de RAG
model = ChatOllama(model="llama3.2:latest")  # Cambiado a llama2 por compatibilidad

RAG_TEMPLATE = """
Eres un asiste de preguntas y respuestas. Usa la informacion recuperada para contestar las preguntas, relacionadas al CADS. Si no sabes la respuesta, solo di que no conoces la respuesta. Usa tres sentencias como maximo para dar una respuesta concisa.

<context>
{context}
</context>

Contesta la siguiente pregunta: 

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

# Definir la función para formatear documentos
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vectorstore.as_retriever()

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)


Finalmente solo quedaria realizar preguntas relacionadas al documento.

In [6]:
# Función para manejar una sola pregunta
async def process_question(question):
    print(f"Procesando pregunta: {question}\n")
    response = ""
    try:
        async for chunk in chain.astream(question):
            response += chunk
            clear_output(wait=True)
            print(f"Pregunta: {question}")
            print(f"Respuesta: {response}", flush=True)
        return response
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Función para ejecutar la pregunta
def run_single_question(question):
    return asyncio.get_event_loop().run_until_complete(process_question(question))

In [7]:
respuesta=run_single_question("De que habla el documento?")

Pregunta: De que habla el documento?
Respuesta: El documento menciona los servicios ofrecidos por CADS, como procesamiento de datos e imágenes, configuración de aplicaciones científicas, desarrollo y optimización de código, modelado de paralelización de software, soluciones para análisis de grandes volúmenes de datos (Big Data) y visualización científica. También se menciona la posibilidad de servicios de consultoría para proyectos de investigación y capacitación.
